In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
import math
import keras

In [ ]:
WINDOW_SIZE=32
X = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
X_test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
y = X[['target_benzene']].copy()
# z = X[['target_benzene']].copy()
X.drop(columns=['target_carbon_monoxide','target_benzene','target_nitrogen_oxides'], inplace=True)
# X['benzene']=z
# X_test['benzene']=benz
print(X.head())
print(y.head())
print(X_test.head())



In [ ]:
y = np.log1p(y)

In [ ]:
X['date_time'] = pd.to_datetime(X['date_time'])
X_test['date_time'] = pd.to_datetime(X_test['date_time'])
# X['co']=y['target_carbon_monoxide'].shift(1, fill_value=2.5)  #AUTO_REGRESSION FEATURES
# X['benzen']=y['target_benzene' ].shift(1, fill_value=12)#AUTO_REGRESSION FEATURES
# X['nitro']=y['target_nitrogen_oxides'].shift(1, fill_value=167.7)#AUTO_REGRESSION FEATURES

# X.loc[0,'ma_co']=2.5
# X.loc[0,'ma_benzen']=12
# X.loc[0,'ma_nitro']=167.7
# for i in range(y.shape[0]-1):
#     X.loc[i+1,'ma_co']=y.loc[max(0,i-WINDOW_SIZE+1):i,'target_carbon_monoxide'].mean() # MOVING AVERAGE FEATURES
#     X.loc[i+1,'ma_benzen']=y.loc[max(0,i-WINDOW_SIZE+1):i,'target_benzene'].mean() # MOVING AVERAGE FEATURES
#     X.loc[i+1,'ma_nitro']=y.loc[max(0,i-WINDOW_SIZE+1):i,'target_nitrogen_oxides'].mean() # MOVING AVERAGE FEATURES

# X['day'] = X['date_time'].dt.day
# X_test['day'] = X_test['date_time'].dt.day
X['dayofweek'] = X['date_time'].dt.dayofweek
X_test['dayofweek'] = X_test['date_time'].dt.dayofweek
# X['q'] = X['date_time'].dt.quarter
# X_test['q'] = X_test['date_time'].dt.quarter
X['hour'] = X['date_time'].dt.hour
X_test['hour'] = X_test['date_time'].dt.hour
X['temp_1h']=X['deg_C']-X['deg_C'].shift(1, fill_value=10)
X['hum_1h']=X['relative_humidity']-X['relative_humidity'].shift(1, fill_value=40)
X_test['temp_1h']=X_test['deg_C']-X_test['deg_C'].shift(1, fill_value=10)
X_test['hum_1h']=X_test['relative_humidity']-X_test['relative_humidity'].shift(1, fill_value=40)
# X['temp_6h']=X['deg_C']-X['deg_C'].shift(6, fill_value=10)
# X['hum_6h']=X['relative_humidity']-X['relative_humidity'].shift(6, fill_value=40)
# X_test['temp_6h']=X_test['deg_C']-X_test['deg_C'].shift(6, fill_value=10)
# X_test['hum_6h']=X_test['relative_humidity']-X_test['relative_humidity'].shift(6, fill_value=40)
# X['sensor_2^2']=X['sensor_2']**2
# X['sensor_3^-1']=X['sensor_3']**-1
# X_test['sensor_2^2']=X_test['sensor_2']**2
# X_test['sensor_3^-1']=X_test['sensor_3']**-1
X["working_hours"] =  X["hour"].isin(np.arange(8, 21, 1)).astype("int")
X["maximum_hours"] =  X["hour"].isin([8, 9, 17, 18, 19, 20]).astype("int")
X["is_weekend"] = (X["date_time"].dt.dayofweek >= 5).astype("int")
X_test["working_hours"] =  X_test["hour"].isin(np.arange(8, 21, 1)).astype("int")
X_test["maximum_hours"] =  X_test["hour"].isin([8, 9, 17, 18, 19, 20]).astype("int")
X_test["is_weekend"] = (X_test["date_time"].dt.dayofweek >= 5).astype("int")
X['ratio_1']=X['sensor_1']/X['sensor_2']
X['ratio_2']=X['sensor_1']/X['sensor_3']
X['ratio_3']=X['sensor_1']/X['sensor_4']
X['ratio_4']=X['sensor_1']/X['sensor_5']
X['ratio_5']=X['sensor_2']/X['sensor_3']
X['ratio_6']=X['sensor_2']/X['sensor_4']
X['ratio_7']=X['sensor_2']/X['sensor_5']
X['ratio_8']=X['sensor_3']/X['sensor_4']
X['ratio_9']=X['sensor_3']/X['sensor_5']
X['ratio_10']=X['sensor_4']/X['sensor_5']
X_test['ratio_1']=X_test['sensor_1']/X_test['sensor_2']
X_test['ratio_2']=X_test['sensor_1']/X_test['sensor_3']
X_test['ratio_3']=X_test['sensor_1']/X_test['sensor_4']
X_test['ratio_4']=X_test['sensor_1']/X_test['sensor_5']
X_test['ratio_5']=X_test['sensor_2']/X_test['sensor_3']
X_test['ratio_6']=X_test['sensor_2']/X_test['sensor_4']
X_test['ratio_7']=X_test['sensor_2']/X_test['sensor_5']
X_test['ratio_8']=X_test['sensor_3']/X_test['sensor_4']
X_test['ratio_9']=X_test['sensor_3']/X_test['sensor_5']
X_test['ratio_10']=X_test['sensor_4']/X_test['sensor_5']
X['change_1']=X['sensor_1']-X['sensor_1'].shift(1,fill_value=0)
X['change_2']=X['sensor_2']-X['sensor_2'].shift(1,fill_value=0)
X['change_3']=X['sensor_3']-X['sensor_3'].shift(1,fill_value=0)
X['change_4']=X['sensor_4']-X['sensor_4'].shift(1,fill_value=0)
X['change_5']=X['sensor_5']-X['sensor_5'].shift(1,fill_value=0)
X_test['change_1']=X_test['sensor_1']-X_test['sensor_1'].shift(1,fill_value=0)
X_test['change_2']=X_test['sensor_2']-X_test['sensor_2'].shift(1,fill_value=0)
X_test['change_3']=X_test['sensor_3']-X_test['sensor_3'].shift(1,fill_value=0)
X_test['change_4']=X_test['sensor_4']-X_test['sensor_4'].shift(1,fill_value=0)
X_test['change_5']=X_test['sensor_5']-X_test['sensor_5'].shift(1,fill_value=0)
X['Dew_point'] = X['deg_C'].apply(lambda x: (17.27 * x) / (237.7 + x)) + X['absolute_humidity'].apply(lambda x: math.log (x) )
X['Partial_pressure'] = (X['deg_C'].apply (lambda x: (237.7 + x) * 286.8) * X['absolute_humidity']) / 100000
X['Saturated_wvd'] = (X['absolute_humidity'] * 100) / X['relative_humidity']
X_test['Dew_point'] = X_test['deg_C'].apply(lambda x: (17.27 * x) / (237.7 + x)) + X_test['absolute_humidity'].apply(lambda x: math.log (x) )
X_test['Partial_pressure'] = (X_test['deg_C'].apply (lambda x: (237.7 + x) * 286.8) * X_test['absolute_humidity']) / 100000
X_test['Saturated_wvd'] = (X_test['absolute_humidity'] * 100) / X_test['relative_humidity']
# X_test['target_carbon_monoxide']=X_1_phase['target_carbon_monoxide']
# X_test['target_benzene']=X_1_phase['target_benzene']
X.loc[:,['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']]=np.log1p(X.loc[:,['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']])
X_test.loc[:,['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']]=np.log1p(X_test.loc[:,['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']])

In [ ]:
X_test = pd.concat([X.iloc[-WINDOW_SIZE:,:], X_test[1:]]) #LAST VALUES FROM TRAINING SET INTO BEGINING OF TEST SET FOR WINDOW PURPOSE
X.drop(columns='date_time', inplace=True)
X_test=X_test.reset_index(drop=True)
X_test.drop(columns='date_time', inplace=True)



scaler = MinMaxScaler(feature_range=(-1,1))
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)
# pca = PCA(n_components=8)
# X = pca.fit_transform(X)
# X_test = pca.transform(X_test)
# X_val, y_val = X[:101].copy(),y[:101].copy()
#X,y = X[:9].copy(),y[:9].copy()


In [ ]:
y= y.shift(-WINDOW_SIZE+1)
X=np.asarray(X).astype('float32')
X_test=np.asarray(X_test).astype('float32')
y=np.asarray(y).astype('float32')


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input,LSTM, Dense, Dropout, Bidirectional, TimeDistributed, Flatten, SimpleRNN, GRU, Conv1D, MaxPooling1D, Reshape
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds

In [ ]:
dataset=tf.keras.preprocessing.timeseries_dataset_from_array(
    X,
    y,
    sequence_length=WINDOW_SIZE,
    batch_size=1


)



# )
test = tf.keras.preprocessing.timeseries_dataset_from_array(
    X_test,
    None,
    sequence_length=WINDOW_SIZE,
    batch_size=1)




In [ ]:
shuffled_dataset=dataset.shuffle(10000,reshuffle_each_iteration=False)
# val_dataset=shuffled_dataset.take(3500) $ For split ~0.5
# dataset=shuffled_dataset.skip(3500)

In [ ]:
#K-FOLD CSV
fold1 = shuffled_dataset.shard(5,0)
fold2 = shuffled_dataset.shard(5,1)
fold3 = shuffled_dataset.shard(5,2)
fold4 = shuffled_dataset.shard(5,3)
fold5 = shuffled_dataset.shard(5,4)
val_dataset1 = fold1
dataset1 = fold2.concatenate(fold3).concatenate(fold4).concatenate(fold5)
val_dataset2 = fold2
dataset2 = fold1.concatenate(fold3).concatenate(fold4).concatenate(fold5)
val_dataset3 = fold3
dataset3 = fold1.concatenate(fold2).concatenate(fold4).concatenate(fold5)
val_dataset4 = fold4
dataset4 = fold1.concatenate(fold2).concatenate(fold3).concatenate(fold5)
val_dataset5 = fold5
dataset5 = fold1.concatenate(fold2).concatenate(fold3).concatenate(fold5)

dataset1=dataset1.batch(32)
val_dataset1=val_dataset1.batch(32)
dataset2=dataset2.batch(32)
val_dataset2=val_dataset2.batch(32)
dataset3=dataset3.batch(32)
val_dataset3=val_dataset3.batch(32)
dataset4=dataset4.batch(32)
val_dataset4=val_dataset4.batch(32)
dataset5=dataset5.batch(32)
val_dataset5=val_dataset5.batch(32)

train = [dataset1,dataset2,dataset3,dataset4,dataset5]
val = [val_dataset1,val_dataset2,val_dataset3,val_dataset4,val_dataset5]


In [ ]:
# dataset=dataset.batch(32)
# val_dataset=val_dataset.batch(32)
test=test.batch(32)

In [ ]:
def RNNModel():


    
    X_input = Input((1,WINDOW_SIZE,33))
    X = Reshape([WINDOW_SIZE,33])(X_input)

#     X = MaxPooling1D(2,1)(X)
#     X = Conv1D(32,3, activation='relu')(X)
    X = LSTM(60,return_sequences=True, recurrent_dropout=0.25)(X)#, kernel_regularizer=tf.keras.regularizers.L2(0.0008), activity_regularizer=tf.keras.regularizers.L2(0.0008)))(X_input)#, kernel_regularizer=tf.keras.regularizers.L2(0.01), activity_regularizer=tf.keras.regularizers.L2(0.01))(X) 

#     X = TimeDistributed(Dense(60, activation='relu'))(X)
    
    X = LSTM(60, recurrent_dropout=0.25, return_sequences=True)(X)#, kernel_regularizer=tf.keras.regularizers.L2(0.0008), activity_regularizer=tf.keras.regularizers.L2(0.0008)))(X)
#     X = TimeDistributed(Dense(60, activation='relu'))(X)
    X = LSTM(60, recurrent_dropout=0.25, return_sequences=False)(X)#, kernel_regularizer=tf.keras.regularizers.L2(0.0008), activity_regularizer=tf.keras.regularizers.L2(0.0008)))(X)
    X = Dense(1, activation='relu')(X)





    model = Model(inputs = X_input, outputs = X, name='RNNModel')
    return model
def RegrModel():
    X_input = Input((WINDOW_SIZE,11))
    X = Conv1D(64,3)(X_input)

    X = Flatten()(X)
    X = Dense(100, activation='tanh')(X)
    X = Dropout(0.2)(X)
    X = Dense(300, activation='tanh')(X)
    X = Dropout(0.2)(X)
    X = Dense(100, activation='tanh')(X)
    X = Dropout(0.2)(X)
    
    X = Dense(1, activation='relu')(X)
    model = Model(inputs = X_input, outputs = X, name='RegrModel')
    return model

In [ ]:
import tensorflow.keras.backend as K
#RMSLE Implementation
def rmsle_custom(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred)) 

In [ ]:
pred=np.zeros((2247,1))
for fold in range(5):
    keras.backend.clear_session()
    rnnmodel = RNNModel()
    checkpoint_filepath = '/kaggle/working/ckpt1'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    rnnmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
              loss=rmsle_custom,#tf.keras.losses.MeanSquaredLogarithmicError(),
              metrics=['mse'])
    keras.backend.clear_session()
    rnnmodel.fit(train[fold], validation_data=val[fold],epochs=40, callbacks=[model_checkpoint_callback])
    rnnmodel.load_weights(checkpoint_filepath)
    pred += rnnmodel.predict(test)/5

In [ ]:
pred = np.expm1(pred)

In [ ]:
benz = pred.copy()

In [ ]:
pred[:100]

In [ ]:
pred[-100:]

In [ ]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')
sub.iloc[:,2]=pred
print(sub.head())
sub=sub.set_index('date_time')
sub.to_csv('rnn_benzene.csv')

In [ ]:
# subs=sub.copy()

In [ ]:
# sub.iloc[:,1]

In [ ]:
# sub.iloc[:,2]=pred
# print(sub.head())

In [ ]:
# subs.shape

In [ ]:
# sub.to_csv('rnn_conv.csv')

In [ ]:
# sub.to_csv('rnn_suffled_02split.csv')